In [ ]:
#----- STRUCTURE GENERATOR -----
# 1. Generates random carbon structures
# 2. Writes them into ase structure files in Random_Carbon_Structures

# --- INPUTS ---
# Structure Parameters

generate_all_defaults = True

# Manually set params if desired
set_number_of_atoms =216
set_density_g_cm_3 = 2
set_minimum_interatomic_distance = 1.76

# These are required but arbitrary, atomic properties are specified in LAMMPS input files
set_atomic_number = 6
set_atomic_mass = 12

# Random Seed
import numpy as np
rng = np.random.default_rng(1) 

# --- STRUCTURE GENERATOR ---
import ase 
from ase.io import write
from pathlib import Path

# Structure Generator
def random_cell_generator(number_of_atoms, density_g_cm_3, minimum_interatomic_distance, atomic_number, atomic_mass, rng):
    
    # Density calculator - ASE uses Angstrom
    number_density = density_g_cm_3 / atomic_mass * 6.022 * 10 ** 23 / (10**24)

    # Cell Properties
    cell_volume = number_of_atoms / number_density
    cell_length = cell_volume ** (1/3)
    cell = np.full(3, cell_length)

    # Identity and position of atoms
    numbers = np.full(number_of_atoms, atomic_number)
    positions = []
    
    data_dir = Path(f"Random_Structures")
    data_dir.mkdir(exist_ok=True)

    # Files are sorted into by atom-number directories
    num_atom_dir = data_dir / f"{number_of_atoms}_atoms"
    num_atom_dir.mkdir(parents = True, exist_ok=True)

    filename = (
    f"{density_g_cm_3:.1f}_gcm"
    )

    file_path = num_atom_dir / filename

    # Check if structure already exists
    if file_path.exists():
        return None
    
    # Sanity Check on density
    mean_separation = (1 / number_density) ** (1/3)
    if minimum_interatomic_distance > 0.99*mean_separation:
        return(print(f"FAILED: Mininum distance too close to mean separation of {mean_separation}"))
    if minimum_interatomic_distance > 0.98*mean_separation:
        print(f"WARNING - SLOW:Minimum distance approaching mean separation of {mean_separation}")
    
    # Rejection Sampling used to generate random positions
    while len(positions) < number_of_atoms:   
        new_atom = rng.random(3) * cell_length
        if len(positions) == 0:
                nearest_neighbour = np.inf
        else:  
            pos_array = np.array(positions)
            vector_diffs = pos_array - new_atom
            scalar_diffs = np.linalg.norm(vector_diffs, axis=1)
            nearest_neighbour = min(scalar_diffs)
        if nearest_neighbour > minimum_interatomic_distance:
            positions.append(new_atom)
            
    # Final Structure
    structure = ase.Atoms(
        numbers=numbers,
        positions=positions,
        cell=cell,
        pbc=True
    )

    write(
        file_path,
        structure,
        format="lammps-data",
        atom_style="atomic" 
    )
    return file_path


if generate_all_defaults:
    number_of_atoms =64
    minimum_interatomic_distance = 1.76
    atomic_number = 14
    atomic_mass = 28.085
    for density_g_cm_3 in np.arange (1.5,3.6,0.1):
        new_file = random_cell_generator(number_of_atoms, density_g_cm_3, 
                                         minimum_interatomic_distance, atomic_number, 
                                         atomic_mass, rng )
        if new_file is None:
            continue
        print(f"{new_file} created")
else:
    new_file = random_cell_generator(set_number_of_atoms, set_density_g_cm_3, 
                          set_minimum_interatomic_distance, set_atomic_number, 
                          set_atomic_mass, rng )
    print(f"{new_file} created")



In [ ]:
# Structure Visualiser

from ase.visualize import view
from ase.io import read

atoms = read("Random_Structures/216_atoms/1.5_gcm", format="lammps-data")
view(atoms, viewer='x3d')
